<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/projects/65_water_detect_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install waterdetect rasterio

In [ ]:
import pooch
import rasterio as rio
import waterdetect as wd

In [ ]:
s2_img = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/main/raster/oromocto_jun2021_composite.tif',
    known_hash='e08df41e0f301d684f9ed1a487b9abccaf431fd3df25fa5f10da82d44e6b733d'
)

In [ ]:
!waterdetect -GC

Copying /usr/local/lib/python3.10/dist-packages/WaterDetect.ini into current dir.
WaterDetect.ini copied into /content.


In [ ]:
config = wd.DWConfig(config_file='WaterDetect.ini')
config.clustering_bands, config.detect_water_cluster

Loading configuration file WaterDetect.ini
File WaterDetect.ini verified.


([['ndwi', 'Nir']], 'maxndwi')

In [ ]:
with rio.open(s2_img) as src:
    green, nir = src.read([2, 7])
bands = {'Green': green / 10000, 'Nir': nir / 10000}
wmask = wd.DWImageClustering(bands=bands, bands_keys=['Nir', 'ndwi'], invalid_mask=None, config=config)
mask = wmask.run_detect_water()

Selection of best number of clusters using Calinski-Harabasz Index:
k=2 :Calinski_harabaz index=21678.1805929731
k=3 :Calinski_harabaz index=39384.42501473059
k=4 :Calinski_harabaz index=26519.44141131418
k=5 :Calinski_harabaz index=19907.481413793612
k=6 :Calinski_harabaz index=19740.91847634046
k=7 :Calinski_harabaz index=24071.988893085832
Applying clusters based naive bayes classifier
Assgnin 1 to cluster_id 1
Skipping cluster_id 1


In [ ]:
with rio.open(s2_img) as src:
    cfg = src.profile | dict(count=1, dtype=mask.dtype)
    with rio.open('mask.tif', 'w', **cfg) as out:
        out.write(mask, 1)